In [10]:
import pandas as pd
import numpy as np
import gzip
import os
import datetime
import pandas as pd
import calendar
import numpy as np
import glob
import sys
import gc
import re
#import quantstats as qs
import time

import matplotlib.pyplot as plt

import nest_asyncio # install this package to avoid running in loop

import urllib
import pandas_ta as ta

import warnings

import requests

import hmac
import hashlib

import aiohttp
import asyncio

import logging
from datetime import timezone

import nest_asyncio

# Filter depreciation warnings from pandas regarding the append method
warnings.filterwarnings('ignore', category=FutureWarning, message=".*append.*")
pd.options.mode.chained_assignment = None

# Set pandas to display all digits for floating-point numbers
pd.options.display.float_format = '{:.8f}'.format

import os
from dotenv import load_dotenv

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Ignore only FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

# Ignore only RuntimeWarning
warnings.filterwarnings('ignore', category=RuntimeWarning)

import importlib


from Orats_options_fetch_functions import *

from faster_numba_strategy_functions import *

import strategy_functions

importlib.reload(strategy_functions)
from strategy_functions import *

# import Common_functions

# importlib.reload(strategy_functions)
# importlib.reload(Common_functions)

# from Common_functions import *
# coin_desk_api_key =  "e43c1fd991e660b9bf959645f0800bb7e76fb4a3537ab773cec62b2fad31af2d"
from datetime import time
from pathlib import Path
# Get the user's home directory dynamically
home_dir = r"C:\Data\Options_logs"
#====================================

strategy_name = "Backtest_v1"

# Create a folder for the strategy_name if it doesn't exist in home directory

strategy_folder = os.path.join(home_dir, strategy_name)
if not os.path.exists(strategy_folder):
    os.makedirs(strategy_folder)

# Create the full path to the output file. this file contains all the trades

output_all_trades_file = os.path.join(strategy_folder, f"All_trades_{strategy_name}.csv")
trade_html_file = os.path.join(strategy_folder, f"Trade_stats_{strategy_name}.html")

#=== location to store the trade dataframe for debug

base_folder_df = "strategy_dataframe"
df_folder = os.path.join(strategy_folder, base_folder_df)
if not os.path.exists(df_folder):
    os.makedirs(df_folder)
    print("\n Created strategy dataframe folder : " ,df_folder )

#----- symbol_wise_trades

trades_folder = "sym_wise_trades"
trades_strategy_folder = os.path.join(strategy_folder, trades_folder)

if not os.path.exists(trades_strategy_folder):
    print("\n Making folder for symbol wise trades of strategy" , trades_strategy_folder)
    os.makedirs(trades_strategy_folder)

logger = setup_logger("Straddle_0DTE", strategy_folder, log_file="strategy.log" , to_console=False)


In [11]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

# Connect to an in-memory database
con = duckdb.connect()

# --- Your setup code (which is correct) ---
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("""
SET s3_endpoint='122.176.133.73:9000';
SET s3_use_ssl=false;
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
""")
con.execute("SET s3_url_style='path';") 
con.execute("PRAGMA threads=8;")
con.execute("PRAGMA enable_object_cache;")

# --- THE MISSING STEP: Querying a file from S3 ---
# Replace with your actual S3 bucket and file path
# This can be a .parquet, .csv, or .json file
s3_file_path = 's3://duckdata/ORATS/Options/**/*.parquet'

try:
    # Execute a query on the S3 file and fetch the result as a Pandas DataFrame
    print(f"Querying data from: {s3_file_path}")
    df = con.execute(f"""
        SELECT *
        FROM '{s3_file_path}'
        LIMIT 2;
    """).df()

    print("\n✅ Query successful! First 5 rows of data:")
    print(df.head())

except Exception as e:
    print(f"\n❌ An error occurred during the query: {e}")

# finally:
#     # It's good practice to close the connection
#     con.close()

Querying data from: s3://duckdata/ORATS/Options/**/*.parquet

✅ Query successful! First 5 rows of data:
      expiry       strike  dte  optionType                  ts         low  \
0 2024-01-26 421.00000000   25          -1 2024-01-02 17:13:00 17.18000000   
1 2024-01-12 414.00000000   11          -1 2024-01-02 17:13:00 10.41500000   

         high        open       close     volume  ...      askSize  \
0 17.18000000 17.18000000 17.18000000 0.00000000  ...  50.00000000   
1 10.41500000 10.41500000 10.41500000 0.00000000  ... 151.00000000   

     bidPrice    askPrice      bidIv      askIv         iv          oi  \
0 17.12000000 17.24000000 0.13785700 0.14511500 0.14148600 15.00000000   
1 10.34000000 10.49000000 0.14267100 0.15199600 0.14733300 54.00000000   

    stockPrice        day ticker  
0 404.00000000 2024-01-02    QQQ  
1 404.00000000 2024-01-02    QQQ  

[2 rows x 21 columns]


In [13]:
df = pd.read_excel(r"C:\Data\Aggregate_Data\US_top_250.xlsx")

In [15]:
df['Symbol'] = df['Symbol'].str.strip()

In [17]:
us_top_250 = df['Symbol'].to_list()

In [18]:
us_top_250

['NVDA',
 'MSFT',
 'AAPL',
 'GOOGL',
 'AMZN',
 'META',
 'AVGO',
 'TSLA',
 'TSM',
 'BRK.B',
 'ORCL',
 'JPM',
 'WMT',
 'LLY',
 'V',
 'MA',
 'NFLX',
 'XOM',
 'JNJ',
 'COST',
 'HD',
 'PLTR',
 'ABBV',
 'BAC',
 'PG',
 'BABA',
 'ASML',
 'CVX',
 'UNH',
 'GE',
 'SAP',
 'KO',
 'TMUS',
 'CSCO',
 'AMD',
 'WFC',
 'TM',
 'PM',
 'MS',
 'GS',
 'NVO',
 'AZN',
 'IBM',
 'NVS',
 'HSBC',
 'CRM',
 'ABT',
 'AXP',
 'BX',
 'LIN',
 'MCD',
 'RTX',
 'T',
 'SHEL',
 'DIS',
 'UBER',
 'CAT',
 'RY',
 'MRK',
 'APP',
 'NOW',
 'PEP',
 'SHOP',
 'VZ',
 'C',
 'ANET',
 'INTU',
 'PDD',
 'BKNG',
 'TMO',
 'MUFG',
 'MU',
 'QCOM',
 'BLK',
 'SONY',
 'GEV',
 'HDB',
 'SCHW',
 'SPGI',
 'BA',
 'ARM',
 'TXN',
 'TJX',
 'ISRG',
 'UL',
 'LOW',
 'LRCX',
 'SAN',
 'ACN',
 'BSX',
 'AMGN',
 'ADBE',
 'NEE',
 'ETN',
 'APH',
 'COF',
 'SPOT',
 'SYK',
 'PGR',
 'GILD',
 'BHP',
 'PFE',
 'AMAT',
 'DHR',
 'PANW',
 'TTE',
 'HON',
 'TD',
 'UBS',
 'KLAC',
 'KKR',
 'UNP',
 'DE',
 'BTI',
 'ADI',
 'CMCSA',
 'MDT',
 'MELI',
 'ADP',
 'COP',
 'INTC',
 'IBN',
 '

In [19]:

#====================================
ifol = r"C:\Data\Aggregate_Data\Correct_US_SP500_2019_2025_15min.parquet"


#ifol = r"C:\Data\Aggregate_Data\US_Stocks_2022_2024\qspark\US_qspark_15_min_US_mid_liquidity_267_stocks_2019_2015.parquet"

com_df2 = pd.read_parquet(ifol)

com_df2 = com_df2.loc[:, ~com_df2.columns.duplicated(keep='first')]

In [31]:
# This is the most important change to fix your issue.
final_symbols = com_df2.filter(like='_close').columns.str.replace('_close', '').unique()


In [32]:
final_symbols

Index(['A', 'AAPL', 'ABBV', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP',
       ...
       'WRB', 'WSM', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZTS'],
      dtype='object', length=502)

In [33]:

symbols_to_keep = [symbol for symbol in final_symbols if symbol in us_top_250]

# Now, len(symbols_to_keep) should be the true unique count (e.g., 174)

# Step 3: Create a list of all columns that should be kept using a more robust check
columns_to_keep = []
for symbol in symbols_to_keep:
    # This is a more robust way to match, preventing 'WMT' from matching 'WMTB'
    cols_for_symbol = [col for col in com_df2.columns if col.startswith(symbol + '_')]
    columns_to_keep.extend(cols_for_symbol)

# Step 4: Create the new DataFrame
filtered_com_df = com_df2[columns_to_keep]

# --- Verification Step ---
# This should now give you the expected count of 174
final_symbols2 = filtered_com_df.filter(like='_close').columns.str.replace('_close', '')
print(f"Length of final symbols: {len(final_symbols2)}")

Length of final symbols: 174


In [35]:
com_df4 = quick_float32_convert2(filtered_com_df)

In [37]:
com_df4.to_parquet(r"C:\Data\Aggregate_Data\US_top_200_2019_2025_15min.parquet")

In [27]:

final_symbols2 = filtered_com_df.filter(like='_close').columns.str.replace('_close', '')

In [28]:
final_symbols2

Index(['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AJG',
       'AMAT',
       ...
       'VZ', 'WELL', 'WFC', 'WM', 'WMB', 'WMT', 'WMB', 'WMT', 'XOM', 'ZTS'],
      dtype='object', length=296)

In [5]:
# --- STRATEGY CONFIGURATION ---
# This dictionary now controls the entire strategy.

# Example 1: 0DTE At-the-Money LONG STRADDLE
strategy_config = {
    'Leg1': {'action': 'SELL', 'option_type': 1, 'expiry_offset': 0, 'strike_offset': 5},  # Buy ATM Call
    'Leg2': {'action': 'BUY', 'option_type': 1, 'expiry_offset': 1, 'strike_offset': 7}   # Buy ATM Put
}

# Example 2: 0DTE SHORT STRANGLE
# strategy_config = {
#     'Leg1': {'action': 'SELL', 'option_type': 1, 'expiry_offset': 0, 'strike_offset': 5}, # Sell OTM Call
#     'Leg2': {'action': 'SELL', 'option_type': 0, 'expiry_offset': 0, 'strike_offset': -5} # Sell OTM Put
# }

# This defines the value of one strike step (e.g., for SPY, strikes are 1 point apart)
strike_interval = 1.0

In [6]:


ifol = r"C:\Data\Aggregate_Data\Options\USA\SPY_1min_2024_2025_ORATS_close.parquet"
#== read the RELIANCE data

com_df2 = pd.read_parquet(ifol)

com_df2 = com_df2.drop_duplicates(keep='first')

com_df2.dropna(how='all', inplace=True)

vix_df = pd.read_parquet(r"C:\Data\Aggregate_Data\VIX_1_min_2019_2025.parquet")
com_df2 = com_df2.join(vix_df)


final_symbols = com_df2.filter(like='close').columns.str.replace('_close', '')
print("Total symbols found after all filtering: ", len(final_symbols))

# Calculate the number of rows in the DataFrame

com_df2[[f"{symbol}_intraday_low" for symbol in final_symbols]] =com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: intraday_low(com_df2, low_col=x.name,agg_func='min'))

com_df2[[f"{symbol}_intraday_high" for symbol in final_symbols]] =com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: intraday_high(com_df2, high_col=x.name,agg_func='max'))

com_df2[[f"{symbol}_todayo" for symbol in final_symbols]] = com_df2[[f'{symbol}_close' for symbol in final_symbols]].resample('D').transform('first')

# com_df2[[f"{symbol}_daily_ROC" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change(periods=50)

# com_df2[[f"{symbol}_ROC" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change()

# com_df2[[f"{symbol}_ROC_4" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].pct_change(periods=4)

com_df2[[f"{symbol}_prevdayl" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_low_numba(com_df2, n=1, column=x.name))

com_df2[[f"{symbol}_prevdayh" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=1, column=x.name))

com_df2[[f"{symbol}_secdayh" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=2, column=x.name))


# This single call does it for the 20-day MA
ma_20d_df = calculate_daily_ma_vectorized(df=com_df2, symbols=final_symbols, periods=3 , agg_func='last' , col_suffix='close')

    # Join the new features back to the main DataFrame
com_df2 = pd.concat([com_df2, ma_20d_df] , axis=1 , join='outer')


com_df2[[f"{symbol}_ma" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(2).mean()

com_df2[[f"{symbol}_ROC_30" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma" for symbol in final_symbols]].pct_change(periods=30)


#com_df2[[f"{symbol}_ma_3d_low" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma" for symbol in final_symbols]].values / com_df2[[f"{symbol}_3d_MA" for symbol in final_symbols]].values

com_df2[[f"{symbol}_prevdayc" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: calculate_prev_day_close(com_df2, close_col=x.name, n_days=1))

com_df2[[f"{symbol}_secdayc" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].apply(lambda x: calculate_prev_day_close(com_df2, close_col=x.name, n_days=2))


# #=== hourly volume
# com_df2[[f"{symbol}_vol_ma" for symbol in final_symbols]] = com_df2[[f"{symbol}_volume" for symbol in final_symbols]].rolling(4).mean()

# #==avg 3 day volume

# com_df2[[f"{symbol}_3d_vol_MA" for symbol in final_symbols]] = com_df2[[f"{symbol}_volume" for symbol in final_symbols]].apply(lambda x: daily_moving_average(com_df2, timeframe='1D', column=x.name, periods=3, agg_func='last'))

com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(5).mean()


com_df2[[f"{symbol}_3dlow" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]].apply(lambda x: get_x_day_low_numba(com_df2, n=3, column=x.name))

com_df2[[f"{symbol}_3dhigh" for symbol in final_symbols]] = com_df2[[f"{symbol}_ma2_low" for symbol in final_symbols]].apply(lambda x: get_x_day_high_numba(com_df2, n=3, column=x.name))



# 1. Get the lists of column names
high_3d_cols = [f"{symbol}_3dhigh" for symbol in final_symbols]
low_3d_cols = [f"{symbol}_3dlow" for symbol in final_symbols]

# 2. Perform the calculation on NumPy arrays
range_values = (com_df2[high_3d_cols].values / com_df2[low_3d_cols].values) - 1

# 3. Create a new DataFrame from the results
# This is the key step to avoid the error.
range_cols = [f"{symbol}_3d_rng" for symbol in final_symbols]
range_df = pd.DataFrame(range_values, index=com_df2.index, columns=range_cols)

# 4. Join the new DataFrame back to the main one
com_df2 = com_df2.join(range_df)



# # Calculate highest high of last 4 bars
# com_df2[[f"{symbol}_high_4bar" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(10).max()

# # Calculate lowest low of last 4 bars
# com_df2[[f"{symbol}_low_4bar" for symbol in final_symbols]] = com_df2[[f"{symbol}_close" for symbol in final_symbols]].rolling(10).min()

# # 1. Get the lists of column names
# high_cols = [f"{symbol}_high_4bar" for symbol in final_symbols]
# low_cols = [f"{symbol}_low_4bar" for symbol in final_symbols]

# # 2. Perform the calculation on the NumPy arrays
# range_values = com_df2[high_cols].values - com_df2[low_cols].values

# # 3. Create a new DataFrame from the result (This is the key step)
# range_cols = [f"{symbol}_range_4bar" for symbol in final_symbols]
# range_df = pd.DataFrame(range_values, index=com_df2.index, columns=range_cols)

# # 4. Join the new DataFrame back to the main one
# com_df2 = com_df2.join(range_df)


# # Calculate mean range (average of the 4-bar ranges)
# com_df2[[f"{symbol}_mean_range_4bar" for symbol in final_symbols]] = com_df2[[f"{symbol}_range_4bar" for symbol in final_symbols]].rolling(12).mean()


# gc.collect()

# for symbol in final_symbols:

   
#     com_df2[f'{symbol}_iATR2'] = com_df2[f"{symbol}_ATR"].values / com_df2[f"{symbol}_close"].values


# 1. Define the column groups
close_cols = [f"{symbol}_close" for symbol in final_symbols]
low_3d_cols = [f"{symbol}_3dlow" for symbol in final_symbols]
high_3d_cols = [f"{symbol}_3dhigh" for symbol in final_symbols]

# 2. Perform the calculation in a more chained and memory-conscious way
# Using .to_numpy(dtype=np.float32) can cut memory usage by 50%
numerator = com_df2[close_cols].to_numpy(dtype=np.float32) - com_df2[low_3d_cols].to_numpy(dtype=np.float32)
denominator = com_df2[high_3d_cols].to_numpy(dtype=np.float32) - com_df2[low_3d_cols].to_numpy(dtype=np.float32)

# np.divide handles division by zero safely
percentile_values = np.divide(numerator, denominator, where=denominator != 0)

# 3. Create the new feature DataFrame
pctl_cols = [f"{symbol}_3d_Pctl" for symbol in final_symbols]
percentile_df = pd.DataFrame(percentile_values, index=com_df2.index, columns=pctl_cols)

# 4. Clip values and join back (same as your code)
percentile_df = percentile_df.clip(0, 1)
com_df2 = com_df2.join(percentile_df)


com_df2['decent'] = np.where((com_df2['SPY_todayo'] < com_df2['SPY_prevdayc'] * 1.005) & (com_df2['SPY_todayo'] > com_df2['SPY_prevdayc'] * 0.98), 1, 0)
com_df = com_df2[com_df2['decent']==1]
com_df = com_df[com_df.index.year>=2024]

Total symbols found after all filtering:  2
Calculating 3-day MA for 2 symbols...


In [ ]:
com_df

In [7]:
import pandas as pd
import pandas_market_calendars as mcal

def fetch_option_data_for_n_days(con, ticker, end_date_str, n_days, strike, expiry_str, option_type):
    """
    Fetches a range of minute-level data for a single option contract
    for a specified number of business days ending on a given date.

    Args:
        con: The DuckDB connection object.
        ticker (str): The stock ticker (e.g., 'SPY').
        end_date_str (str): The last day to query, in 'YYYY-MM-DD' format.
        n_days (int): The number of business days of data to fetch.
        strike (float): The specific strike price.
        expiry_str (str): The expiry date to filter for, in 'YYYY-MM-DD' format.
        option_type (int): The type of option to fetch (1 for Calls, 0 for Puts).

    Returns:
        pd.DataFrame: A DataFrame containing the minute-by-minute data.
    """
    option_name = "Call" if option_type == 1 else "Put"
    #print(f"Fetching last {n_days} b-days of data ending on {end_date_str} for {ticker} {strike} {option_name} expiring {expiry_str}...")
    
    try:
        # --- Business Day Calculation Logic ---
        # 1. Get the market calendar
        nyse = mcal.get_calendar('NYSE')
        
        # 2. To be safe, find a start date far enough in the past
        #    (n * 2 is a safe buffer for weekends/holidays)
        start_buffer = pd.to_datetime(end_date_str) - pd.Timedelta(days=n_days * 2)
        
        # 3. Generate a schedule of valid trading days
        schedule = nyse.schedule(start_date=start_buffer, end_date=end_date_str)
        
        # 4. Get the last 'n_days' from the schedule and format them
        business_day_list = [d.strftime('%Y-%m-%d') for d in schedule.index[-n_days:]]
        
        # --- Query Logic (remains the same) ---
        path_list = [
            f"'s3://duckdata/ORATS/Options/ticker={ticker}/day={d}/*.parquet'"
            for d in business_day_list
        ]
        
        query = f"""
            SELECT ts, strike, expiry, dte, optionType, volume, oi, close,
                bidPrice, askPrice, iv, stockPrice, ticker
            FROM read_parquet([{",".join(path_list)}])
            WHERE 
                strike = {strike} AND
                expiry = '{expiry_str}' AND
                optionType = {option_type}
            ORDER BY ts;
        """
        
        multi_day_df = con.execute(query).df()
        
        if multi_day_df.empty:
            print("INFO: No data found for this contract in the specified date range.")
        
        multi_day_df['ts'] = pd.to_datetime(multi_day_df['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None))
        multi_day_df.set_index('ts' , inplace=True)
        return multi_day_df

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# ==============================================================================
# --- Example Usage ---
# ==============================================================================

# Assume 'con' is your active DuckDB connection

# Get data for the 5 business days ending on September 12, 2025
multi_day_df = fetch_option_data_for_n_days(
    con=con,
    ticker='SPY',
    end_date_str='2024-04-12', # The last day of the range
    n_days=3,                  # How many business days to go back
    strike=510.0,
    expiry_str='2024-04-12',
    option_type=1              # 1 for Call
)

if not multi_day_df.empty:
    print("\n--- Found Data for Date Range ---")
    print(multi_day_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


--- Found Data for Date Range ---
                          strike     expiry  dte  optionType        volume  \
ts                                                                           
2024-04-10 09:30:00 510.00000000 2024-04-12    3           1 -700.00000000   
2024-04-10 09:31:00 510.00000000 2024-04-12    3           1    8.00000000   
2024-04-10 09:32:00 510.00000000 2024-04-12    3           1    0.00000000   
2024-04-10 09:33:00 510.00000000 2024-04-12    3           1   36.00000000   
2024-04-10 09:34:00 510.00000000 2024-04-12    3           1    8.00000000   
...                          ...        ...  ...         ...           ...   
2024-04-12 15:56:00 510.00000000 2024-04-12    1           1 1958.00000000   
2024-04-12 15:57:00 510.00000000 2024-04-12    1           1  695.00000000   
2024-04-12 15:58:00 510.00000000 2024-04-12    1           1  604.00000000   
2024-04-12 15:59:00 510.00000000 2024-04-12    1           1  609.00000000   
2024-04-12 16:00:00 510.00000

In [8]:
def fetch_option_data_for_n_days_MOST_RELIABLE(con, ticker, end_date_str, n_days, strike, expiry_str, option_type):
    try:
        # --- Use mcal for the most accurate date calculation ---
        nyse = mcal.get_calendar('NYSE')
        schedule = nyse.schedule(start_date=pd.to_datetime(end_date_str) - pd.Timedelta(days=n_days * 2), end_date=end_date_str)
        if len(schedule) < n_days:
            raise ValueError("Not enough historical trading days in the calendar for the requested n_days.")
        
        # Get the precise start date from the calendar
        start_date_str = schedule.index[-n_days].strftime('%Y-%m-%d')

        # --- Use the fast, optimized DuckDB query ---
        s3_path = f's3://duckdata/ORATS/Options/ticker={ticker}/day=*/*.parquet'
        query = f"""
            SELECT ts, strike, expiry, dte, optionType, volume, oi, close,
                bidPrice, askPrice, iv, stockPrice, ticker
            FROM read_parquet('{s3_path}', hive_partitioning = 1)
            WHERE day >= '{start_date_str}' AND day <= '{end_date_str}'
              AND strike = {strike} AND expiry = '{expiry_str}' AND optionType = {option_type}
            ORDER BY ts;
        """
        
        
        multi_day_df = con.execute(query).df()

        if multi_day_df.empty:
            print("INFO: No data found for this contract in the specified date range.")
        
        multi_day_df['ts'] = pd.to_datetime(multi_day_df['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None))
        multi_day_df.set_index('ts' , inplace=True)
        return multi_day_df

        # ... (rest of the function remains the same) ...

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

In [ ]:
com_df.index.normalize().unique()

In [ ]:

# Get data for the 5 business days ending on September 12, 2025
multi_day_df = fetch_option_data_for_n_days_MOST_RELIABLE(
    con=con,
    ticker='SPY',
    end_date_str='2024-09-12', # The last day of the range
    n_days=3,                  # How many business days to go back
    strike=510.0,
    expiry_str='2024-09-12',
    option_type=1              # 1 for Call
)



In [ ]:
import timeit 
# --- 2. Test Parameters ---
test_dates = pd.to_datetime([
    '2024-09-12', '2024-09-13', '2024-09-16', '2024-09-17', '2024-09-18'
])
test_params = {
    'ticker': 'SPY',
    'n_days': 5,
    'strike': 520.0,
    'expiry_str': '2024-09-19',
    'option_type': 1
}

original_times = []
reliable_times = []


# --- 3. Loop through dates and time each function ---
for date in test_dates:
    date_str = date.strftime('%Y-%m-%d')
    print(f"\nTesting for end_date: {date_str}")


    # --- Time the MOST_RELIABLE function using timeit.default_timer() ---
    print("  -> Timing MOST_RELIABLE (Hive) function...")
    start_time = timeit.default_timer()
    df2 = fetch_option_data_for_n_days_MOST_RELIABLE(con=con, end_date_str=date_str, **test_params)
    duration = timeit.default_timer() - start_time
    reliable_times.append(duration)
    print(f"     Done in {duration:.4f} seconds. Found {len(df2)} rows.")



    
    # --- Time the ORIGINAL function using timeit.default_timer() ---
    print("  -> Timing Original function...")
    start_time = timeit.default_timer()
    df1 = fetch_option_data_for_n_days(con=con, end_date_str=date_str, **test_params)
    duration = timeit.default_timer() - start_time
    original_times.append(duration)
    print(f"     Done in {duration:.4f} seconds. Found {len(df1)} rows.")
# --- 4. Print Summary ---
avg_original = np.mean(original_times)
avg_reliable = np.mean(reliable_times)

print("\n--- ⏱️ BENCHMARK COMPLETE ---")
print(f"🐢 Original Function Average Time:   {avg_original:.4f} seconds")
print(f"🚀 MOST_RELIABLE Function Average Time: {avg_reliable:.4f} seconds")



In [ ]:
strike_interval = 1
lookback_days = 3
option_multiplier = 100
from collections import  defaultdict

# ==============================================================================
# SCRIPT STARTS HERE
# ==============================================================================

trade_log = []
nyse = mcal.get_calendar('NYSE') 
unique_days = com_df.index.normalize().unique()
print(f"🚀 Starting backtest for {len(unique_days)} unique days...")

intial_capital = 100000
position_cap = 100000

# Track local trade log
local_trade_log2 = pd.DataFrame(columns=['name', 'Side', 'entry_date', 'entry_price' , 'qty' ,  'exit_date', 'exit_price', 'order_no','profit' ])

# ✅ Initialize a list to hold all trade dictionaries.
final_trade_log_list = []

# --- Main Loop: Iterate Through Each Day ---
for day in unique_days[2:7]:
    
    day_str = day.strftime('%Y-%m-%d')
    print(f"\n--- Processing Day: {day_str} ---")

    try:
        
        daily_underlying_df = com_df[com_df.index.date == day.date()]
        if daily_underlying_df.empty: continue

        open_price = daily_underlying_df['SPY_close'].iloc[0]
        if pd.isna(open_price): continue
        
        atm_strike = np.round(open_price)
        #print(f"  STEP 1: ATM Strike for the day set to {atm_strike}")

        # --- Dynamically Build and Fetch Data Based on Config ---
        all_legs_hist_data = {}
        fetch_success = True
        legs_to_trade = []

        for leg_name, params in strategy_config.items():
            
            target_strike = atm_strike + (params['strike_offset'] * strike_interval)

            print(f"Atm strike is {atm_strike} and taegt strike is {target_strike} and optiontyoe is {params['option_type']}")
            schedule = nyse.schedule(start_date=day, end_date=day + pd.Timedelta(days=14))
            target_expiry_str = schedule.index[params['expiry_offset']].strftime('%Y-%m-%d')
            print(f"Traget expiry for {leg_name} and {day_str} is {target_expiry_str} ")
            legs_to_trade.append({
                'name': leg_name, 'strike': target_strike,
                'expiry': target_expiry_str, 'type': params['option_type']})
            
            leg_hist_df = fetch_option_data_for_n_days(
                con=con, ticker='SPY', end_date_str=day_str, n_days=lookback_days,
                strike=target_strike, expiry_str=target_expiry_str, option_type=params['option_type'] )
                    
            # ✅ ADD THIS LINE TO DEBUG:
            #print(f"Columns found for {leg_name}: {leg_hist_df.columns.tolist()}")

            if not leg_hist_df.empty:
                # Find rows where both bidPrice and askPrice are 0
                zero_rows = leg_hist_df[ (leg_hist_df['bidPrice'] == 0) & (leg_hist_df['askPrice'] == 0) ]
                zero_count = len(zero_rows)
                total_rows = len(leg_hist_df)
                
                #print(f"  📊 {leg_name}: {zero_count}/{total_rows} rows have both bid & ask = 0")
                # One-liner: Replace zeros with NaN and forward fill for both columns
                leg_hist_df[['bidPrice', 'askPrice']] = leg_hist_df[['bidPrice', 'askPrice']].replace(0, np.nan).ffill()
                
                if zero_count > 1:
                    print(f"  ⚠️  {zero_count} Rows with zero prices for {leg_name}:")

                    if leg_hist_df.empty:
                        print(f"  ⚠️ Could not fetch data for {leg_name}. Skipping day.")
                        fetch_success = False
                        break
                all_legs_hist_data[leg_name] = leg_hist_df
            else:
                print(f"No options data found for {date_str}")
        if not fetch_success: 
            print(f"Not able to fetch options for {day} ")
            continue

        # --- Merge and Prepare Data ---
        prefixed_dfs_today = []
        for leg_name, leg_hist_df in all_legs_hist_data.items():
            
            prefixed_dfs_today.append(leg_hist_df.add_prefix(f'{leg_name}_'))
       
        # 1. First, concatenate all the prefixed option leg DataFrames together
        com_legs_df = pd.concat(prefixed_dfs_today, axis=1, join='outer')
        com_legs_df = com_legs_df[com_legs_df.index.time>time(9 , 30)]
        
        if ( len(com_legs_df)<10):
            print("\n\n =============")
            print(f"Leg data very less for {date_str}")


        #==== now do all calculations and indicators on this
        legs_to_process = ['Leg1', 'Leg2']
                
        com_legs_df[[f"{legs}_intraday_low" for legs in legs_to_process]] = com_legs_df[[f"{legs}_bidPrice" for legs in legs_to_process]].apply(lambda x: intraday_low(com_legs_df, low_col=x.name,agg_func='min'))

        com_legs_df[[f"{legs}_intraday_high" for legs in legs_to_process]] = com_legs_df[[f"{legs}_askPrice" for legs in legs_to_process]].apply(lambda x: intraday_high(com_legs_df, high_col=x.name,agg_func='max'))


        # Calculate Today's Opening Price for each leg
        com_legs_df[[f'{leg}_todayo' for leg in legs_to_process]] =  com_legs_df[[f'{leg}_close' for leg in legs_to_process]].resample('D').transform('first')

        # Calculate Previous Day's Low (using bidPrice) for each leg
        com_legs_df[[f"{leg}_prevdayl" for leg in legs_to_process]] = com_legs_df[[f"{leg}_bidPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_low_numba(com_legs_df, n=1, column=x.name))

        # Calculate Previous Day's High (using askPrice) for each leg
        com_legs_df[[f"{leg}_prevdayh" for leg in legs_to_process]] = \
            com_legs_df[[f"{leg}_askPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_high_numba(com_legs_df, n=1, column=x.name) )

        close_cols = [f'{leg}_close' for leg in legs_to_process]
        window_size = 15

        # # 1. Rolling 15-Bar Highest Close
        # com_legs_df[[f'{leg}_15min_high' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).max()

        # # 2. Rolling 15-Bar Lowest Close
        # com_legs_df[[f'{leg}_15min_low' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).min()

        # # 3. Rolling 15-Bar Mean Close
        # com_legs_df[[f'{leg}_15min_mean' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).mean()

        com_legs_df[[f'{leg}_ROC_5' for leg in legs_to_process]] = com_legs_df[[f"{leg}_close" for leg in legs_to_process]].pct_change(periods=5)
#=== merge underlying back with com_legs_df

        com_df3 = pd.concat([daily_underlying_df , com_legs_df] , join='outer' , axis=1)

        # --- FIX: Add this line to filter for the current day ---
        com_df3 = com_df3[com_df3.index.date == pd.to_datetime(day_str).date()]

         # Initialize state for this day's trading (if needed)
        instrument_positions = {leg: False for leg in legs_to_process}
        entry_prices = {leg: None for leg in legs_to_process}
        # ... (add any other state variables you need: qty, entry_dates, etc.) ...
       
        entry_dates = {leg: None for leg in legs_to_process}
                
        # To track the current drawdown value for each active leg
        drawdown = {leg: None for leg in legs_to_process}

        # To track P&L for each leg (can be redundant if using series)
        symbol_pnl = {leg: None for leg in legs_to_process}

        # To track the maximum drawdown seen for each leg
        max_drawdown = {leg: 0 for leg in legs_to_process}

        # To count how many bars an active position has been held
        bar_count = {leg: 0 for leg in legs_to_process}

        # To store the quantity/number of contracts for each leg
        qty = {leg: 0 for leg in legs_to_process}

        # To store an order identifier for each leg's trade
        order_no = {leg: 0 for leg in legs_to_process}

        # To store the long/short direction (e.g., 1 for long, -1 for short)
        position_type = {leg: 0 for leg in legs_to_process}

        # To store the drawdown history for each leg
        drawdown_series = {leg: [] for leg in legs_to_process}

        # To store the P&L history for each leg
        symbol_pnl_series = {leg: [] for leg in legs_to_process}

        # To store another P&L history (can be consolidated with the one above)
        absolute_symbol_pnl_series = {leg: [] for leg in legs_to_process}
        # ===================================================================
        # --- Bar-by-Bar Simulation Loop using for i in range() ---
        # ===================================================================
        daily_trade = {'legs': {}}
        print("  STEP 3: Starting bar-by-bar simulation...")
        
        logger.info(f"Running analysis on: {intial_capital} of capital")
        cumulative_realized_pnl = 0.0


        # === Exposure & portfolio trackers ===
        exposure = 0.0        # net exposure
        gross_exposure = 0.0  # absolute exposure

        exposure_series = {}          # net exposure at each timestamp
        gross_exposure_series = {}    # absolute exposure
        portfolio_value_series = {}   # portfolio equity curve
        
        # Initialize before main loop
        total_pnl_series = defaultdict(float)
        # Process each row in time order

        for i in range(len(com_df3)):

            row = com_df3.iloc[i]
            timestamp = com_df3.index[i]

            exposure = 0.0
            gross_exposure = 0.0
            # Initialize the unrealized P&L for the current bar to zero
            unrealized_pnl_for_this_bar = 0.0

            if i == 0:
                in_position = 0
                current_pnl = 0
                exposure_series[timestamp] = 0.0
                gross_exposure_series[timestamp] = 0.0
                portfolio_value_series[timestamp] = intial_capital  # starting capital      
                
                total_pnl_series[timestamp] = 0.0  

            else:
                prev_timestamp = com_df3.index[i - 1]
                
                #=== long and short position based on previous bar

                # Get previous PnL value
                current_pnl = total_pnl_series.get(prev_timestamp, 0.0)


            for legs in legs_to_process:
               
                if f'{legs}_close' not in row:

                        #logger.warning(f"Data not found for {symbol} at {timestamp}")
                        continue  # Skip if data for this instrument is missing
                # Fetch the instrument's specific close price
                close_price = row[f'{legs}_close']
                in_position = instrument_positions.get(legs , False)

                if not in_position:
                    #  and com_df3["VIX_close"].iloc[i-1]<com_df3["VIX_prevdayc"].iloc[i - 1]*0.98
                    # and com_df3["VIX_close"].iloc[i-1]<com_df3["VIX_prevdayc"].iloc[i - 1]*1.02
                    if( time(9 , 55)<= timestamp.time()<=time(13 , 15) and legs=='Leg1' and row[f"{legs}_close"]<1.5 and row[f"{legs}_close"]>0.35    and com_df3[f"SPY_3d_Pctl"].iloc[i-1]<0.8 and  com_df3[f"SPY_3d_Pctl"].iloc[i-1]>0.2  and row[f"{legs}_close"]<min((com_df3[f"{legs}_intraday_high"].iloc[i - 1]*0.9) , com_df3[f"{legs}_close"].iloc[i - 1]*0.999 )  and com_df3[f"{legs}_ROC_5"].iloc[i - 1]<0.1  ) :        

                        com_df3.loc[timestamp, f'{legs}_action'] = 'sell'
                        com_df3.loc[timestamp, f'{legs}_trade_price'] = row[f"{legs}_close"]
                        com_df3.loc[timestamp, f'{legs}_qty'] = 10000
                        com_df3.loc[timestamp, f'{legs}_entry_price'] = row[f"{legs}_close"]
                        com_df3.loc[timestamp, f'{legs}_position'] = -1
                        instrument_positions[legs] = True  # Track that this PT_address is now in position
                        entry_prices[legs] = com_df3.loc[timestamp, f'{legs}_entry_price']
                        #=== we note the hedge entry price and qty for the hedge position
                        
                        #==============================================================
                        position_type[legs] = -1
                        entry_dates[legs] = timestamp
                        bar_count[legs] = 0
                        qty[legs] = np.floor(position_cap/entry_prices[legs])
                        
                        print(f"Short entry in {legs} at entry price {entry_prices[legs]} for qty: {qty[legs]} at {timestamp}")
                        #print(f"Symbol rank is :{row[f"{symbol}_close_S1_ratio_Rank"]} and VIX is : {com_df["VIX_close"].iloc[i]}  " )

                if position_type[legs]<0 and time(15 , 45) < timestamp.time() and entry_prices[legs]>=0    :
                     #print("\n Short Stop loss found for entry: " , entry_prices[symbol])
                    com_df3.loc[timestamp, f'{legs}_action'] = 'sell'
                    com_df3.loc[timestamp, f'{legs}_trade_price'] = close_price
                    com_df3.loc[timestamp, f'{legs}_exit_price'] = close_price
                    profit = (entry_prices[legs] -com_df3.loc[timestamp, f'{legs}_exit_price'] ) * qty[legs]
                    com_df3.loc[timestamp, f'{legs}_profit'] = profit
                    com_df3.loc[timestamp, f'{legs}_position'] = 0  # Exit position
                    instrument_positions[legs] = False  # Remove from active positions
                    bar_count[legs] = 0
                    order_no[legs] = -13

                    profit = (entry_prices[legs] - close_price) * qty[legs] # Example for a long trade
                    
                    # 2. Append a dictionary to the list
                    final_trade_log_list.append({
                        'name': legs,
                        'Side': 'sell',
                        'entry_date': entry_dates[legs],
                        'entry_price': entry_prices[legs],
                        'qty': qty[legs],
                        'exit_date': timestamp,
                        'exit_price': close_price,
                        'profit': profit
                    })

                    # ✅ Reset entry price after selling
                    entry_prices[legs] = None
                    #in_position = 0
                    position_type[legs]=0
                
                if position_type[legs]<0 and time(10 , 5) < timestamp.time() and  row[f"{legs}_close"]>entry_prices[legs]+0.5    :
                    print("\n Short Stop loss found for entry: " , entry_prices[legs])
                    com_df3.loc[timestamp, f'{legs}_action'] = 'sell'
                    com_df3.loc[timestamp, f'{legs}_trade_price'] = close_price
                    com_df3.loc[timestamp, f'{legs}_exit_price'] = close_price
                    profit = (entry_prices[legs] -com_df3.loc[timestamp, f'{legs}_exit_price'] ) * qty[legs]
                    com_df3.loc[timestamp, f'{legs}_profit'] = profit
                    com_df3.loc[timestamp, f'{legs}_position'] = 0  # Exit position
                    instrument_positions[legs] = False  # Remove from active positions
                    bar_count[legs] = 0
                    order_no[legs] = -13

                    profit = (entry_prices[legs] - close_price) * qty[legs] # Example for a long trade
                    
                    # 2. Append a dictionary to the list
                    final_trade_log_list.append({
                        'name': legs,
                        'Side': 'sell',
                        'entry_date': entry_dates[legs],
                        'entry_price': entry_prices[legs],
                        'qty': qty[legs],
                        'exit_date': timestamp,
                        'exit_price': close_price,
                        'profit': profit
                    })

                    # ✅ Reset entry price after selling
                    entry_prices[legs] = None
                    #in_position = 0
                    position_type[legs]=0    

    except Exception as e:
        print(f"  ❌ An error occurred on {day_str}: {e}")
    

# (Final results analysis)
log_df = pd.DataFrame(final_trade_log_list)
print("\n--- BACKTEST COMPLETE ---")
if not log_df.empty:
    print(log_df)

In [ ]:
local_trade_log2 = log_df

In [ ]:
log_df

In [ ]:
if 'entry_date' in local_trade_log2.columns:

    local_trade_log2.set_index('entry_date' , inplace=True)

    local_trade_log2.sort_index( ascending=True, inplace=True)

if 'exit_date' in local_trade_log2.columns:
    local_trade_log2['exit_date'] = pd.to_datetime(local_trade_log2['exit_date'])

# Calculate the absolute percentage difference between entry and exit prices
local_trade_log2['price_change_pct'] = (local_trade_log2['exit_price'] - local_trade_log2['entry_price']).abs() / local_trade_log2['entry_price']

# Count trades before filtering
trades_before_filtering = len(local_trade_log2)

average_trade_profit = local_trade_log2['profit'].mean()

# First ensure consistent case for Side column
local_trade_log2['Side'] = local_trade_log2['Side'].str.lower()

# Calculate profit in bps (handles both buy/long and sell/short)
local_trade_log2['profit_bps'] = np.where(
    local_trade_log2['Side'].isin(['buy', 'long']),
    ((local_trade_log2['exit_price'] - local_trade_log2['entry_price']) / local_trade_log2['entry_price']) * 10000,
    ((local_trade_log2['entry_price'] - local_trade_log2['exit_price']) / local_trade_log2['entry_price']) * 10000)

# open_positions = list(long_position_count.values())
# print(f"Max open long positions are: {np.max(open_positions)}")

# open__short_positions = list(short_position_count.values())
# print(f"Max open shrt positions are: {np.max(open__short_positions)}")

total_trade_count = len(local_trade_log2)
max_entries_per_day = local_trade_log2.groupby(local_trade_log2.index.date).size().max()

# Identify losing trades and percentage

losers = local_trade_log2[local_trade_log2['profit'] < 0]
num_losing_trades = len(losers)
losers_percentage = (num_losing_trades / total_trade_count) * 100 if total_trade_count > 0 else 0

winners = local_trade_log2[local_trade_log2['profit'] > 0]

# 1. Create a pandas Series from your PnL data
pnl_series = pd.Series(total_pnl_series)

# 2. Find the minimum value in the series
max_unrealized_loss = pnl_series.min()

print(f"The highest unrealized loss (minimum PnL) was: ${max_unrealized_loss:,.2f}")

# Updated side checks throughout
num_long_trades = len(local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])])
num_short_trades = len(local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])])
print(f"Number of long trades: {num_long_trades} and short trades: {num_short_trades} out of total trades: {total_trade_count}")

# Updated profit calculations
avg_long_profit_bps = local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])]['profit_bps'].mean()
avg_short_profit_bps = local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])]['profit_bps'].mean()
print(f"Average long trade profit: {avg_long_profit_bps:.2f} bps and short: {avg_short_profit_bps:.2f} bps")

profit_total_long = local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])]['profit'].sum()
profit_total_short = local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])]['profit'].sum()
print(f"Net long profit: {profit_total_long:.2f} and short profit: {profit_total_short:.2f}")

# Compute holding period
local_trade_log2['holding_days'] = (local_trade_log2['exit_date'] - local_trade_log2.index).dt.days

# Compute equity curve (EC)
local_trade_log2['EC'] = (position_cap*100) + local_trade_log2['profit'].cumsum()

#============================================

# The index should be the 'exit_date' because that's when the equity changes
equity_at_exit = pd.Series(data=local_trade_log2['EC'].values, index=pd.to_datetime(local_trade_log2['exit_date']))

print(f"Equity at exit is : {equity_at_exit}")

# Print average trade profit
print(f"Average Trade Profit: {average_trade_profit:.2f}")

# Print profit in basis points (bps)
print("Profit (bps) for each trade:")
print(f"Avergae trade profit in bps : {local_trade_log2['profit_bps'].mean()}" )

# Print total trade count
print(f"Total Trade Count: {total_trade_count}")

# Print max entries per day
print(f"Max Entries Per Day: {max_entries_per_day}")

# Print number of losing trades
print(f"Number of Losing Trades: {num_losing_trades}")

print(f"Losing Trades Percentage: {losers_percentage:.2f}%")

# Print holding period
print("Avg Holding Period in (Days) :")
print(local_trade_log2['holding_days'].mean())

# Add: Number of trades with loss greater than 10% (i.e. -1000 bps)
loss_threshold_bps = -1000  # -10%
deep_losers = local_trade_log2[local_trade_log2['profit_bps'] < loss_threshold_bps]
num_deep_losers = len(deep_losers)

print(f"Number of Losing Trades > 10% Loss: {num_deep_losers}")

# Add: Average loss of all losing trades
avg_losing_trade = losers['profit'].mean()
print(f"Average Loss per Losing Trade: {avg_losing_trade:.4f}")

# Calculate average profit percentage (in % terms)
average_profit_percent = local_trade_log2['profit'].mean() / position_cap * 100
print(f"\nAverage Profit Percentage per Trade: {average_profit_percent:.2f}%")

# Calculate Trade Expectancy (in monetary terms)
win_rate = len(winners) / total_trade_count
avg_win = winners['profit'].mean()
avg_loss = losers['profit'].mean()
trade_expectancy = (win_rate * avg_win) - ((1 - win_rate) * abs(avg_loss))
print(f"Trade Expectancy: ${trade_expectancy:.2f} per trade")

# Calculate Risk-Adjusted Metrics
risk_reward_ratio = abs(avg_win / avg_loss) if avg_loss != 0 else float('inf')
print(f"Risk-Reward Ratio: {risk_reward_ratio:.2f}:1")


# Add: Average loss of all losing trades
avg_winning_trade = winners['profit'].mean()
print(f"Average Loss per Losing Trade: {avg_winning_trade:.4f}")


abs_profit_df =  pd.DataFrame()
for symbol , data in absolute_symbol_pnl_series.items():

    tdf = pd.DataFrame(data , columns=['timestamp' , symbol])
    tdf.set_index('timestamp' , inplace=True)

    # Reindex to match df_index, filling missing values with 0
    abs_profit_df[symbol] = tdf.reindex(com_df2.index, fill_value=0)

max_unrealised_drawdown = abs_profit_df.sum(axis=1).min()
print(f"Max unrealised loss ( which is unrealised drawdown on notional) is {max_unrealised_drawdown}")


# Long/Short specific metrics
# Update all calculations to use the new condition
long_trades = local_trade_log2[local_trade_log2['Side'].isin(['buy', 'long'])]
short_trades = local_trade_log2[local_trade_log2['Side'].isin(['sell', 'short'])]

num_long_trades = len(long_trades)
num_short_trades = len(short_trades)

# Profit metrics
avg_long_profit_bps = long_trades['profit_bps'].mean()
avg_short_profit_bps = short_trades['profit_bps'].mean()
profit_total_long = long_trades['profit'].sum()
profit_total_short = short_trades['profit'].sum()

# Win/loss metrics
def calculate_consecutive_wins_losses(trades):
    trades = trades.sort_index()
    consecutive = []
    current_streak = 0
    prev_result = None

    for profit in trades['profit']:
        current_result = 'win' if profit > 0 else 'loss'
        if current_result == prev_result or prev_result is None:
            current_streak += 1
        else:
            consecutive.append((prev_result, current_streak))
            current_streak = 1
        prev_result = current_result

    if current_streak > 0:
        consecutive.append((prev_result, current_streak))

    return pd.DataFrame(consecutive, columns=['type', 'length'])

# Calculate for long and short separately
long_consecutive = calculate_consecutive_wins_losses(long_trades)
short_consecutive = calculate_consecutive_wins_losses(short_trades)

def print_streak_stats(name, df):
    if len(df) > 0:
        max_win_streak = df[df['type'] == 'win']['length'].max()
        max_loss_streak = df[df['type'] == 'loss']['length'].max()
        avg_win_streak = df[df['type'] == 'win']['length'].mean()
        avg_loss_streak = df[df['type'] == 'loss']['length'].mean()
        print(f"\n{name} Streaks:")
        print(f"Max Winning Streak: {max_win_streak}")
        print(f"Max Losing Streak: {max_loss_streak}")
        print(f"Avg Winning Streak: {avg_win_streak:.1f}")
        print(f"Avg Losing Streak: {avg_loss_streak:.1f}")

print_streak_stats("Long", long_consecutive)
print_streak_stats("Short", short_consecutive)

# Winning ratios
long_winners = len(long_trades[long_trades['profit'] > 0])
short_winners = len(short_trades[short_trades['profit'] > 0])

long_win_ratio = long_winners / num_long_trades if num_long_trades > 0 else 0
short_win_ratio = short_winners / num_short_trades if num_short_trades > 0 else 0

print(f"\nLong Trades Win Ratio: {long_win_ratio:.2%}")
print(f"\nShort Trades Win Ratio: {short_win_ratio:.2%}")

# Print all metrics
print("\n==========================================")
print(f"Number of long trades: {num_long_trades}")
print(f"Number of short trades: {num_short_trades}")
print(f"Total trades: {total_trade_count}")

print(f"\nAverage long trade profit: {avg_long_profit_bps:.2f} bps")
print(f"Average short trade profit: {avg_short_profit_bps:.2f} bps")
print(f"Net long profit: {profit_total_long:.2f}")
print(f"Net short profit: {profit_total_short:.2f}")

import matplotlib.pyplot as plt

# Scatter plot of trades
plt.figure(figsize=(12, 6))

# X-axis: trade index (or entry date if you want timeline)
x = range(len(local_trade_log2))

# Y-axis: profit
y = local_trade_log2['profit']

# Scatter: green for winners, red for losers
colors = ['green' if p > 0 else 'red' for p in y]

plt.scatter(x, y, c=colors, alpha=0.7, edgecolors='k')

# Add horizontal line at 0 (break-even)
plt.axhline(0, color='black', linestyle='--', linewidth=1)

plt.title("Scatter of Trade Profits")
plt.xlabel("Trade Index")
plt.ylabel("Profit ($)")
plt.show()



from IPython.display import HTML
import matplotlib.pyplot as plt
import base64
from io import BytesIO

# Create equity curve plot
plt.figure(figsize=(10, 6))
local_trade_log2['EC'].plot(title='Equity Curve', grid=True)
plt.ylabel('Portfolio Value')
plt.tight_layout()

# Save plot to temporary buffer
buffer = BytesIO()
plt.savefig(buffer, format='png')
plt.close()
plot_data = base64.b64encode(buffer.getvalue()).decode('utf-8')


#=== symbols specific

# 2. Group by the symbol name and calculate the sum of profits for each.
symbol_profits = local_trade_log2.groupby('name')['profit'].sum()

# Display the total profit for each symbol
print("--- Total Profit per Symbol ---")
print(symbol_profits)


# 3. Sort the results to easily find the best and worst.
sorted_symbols = symbol_profits.sort_values(ascending=False)


# 4. Extract the best and worst symbols from the sorted list.
# The best symbol is the first item (index 0).
best_symbol = sorted_symbols.index[0]
best_profit = sorted_symbols.iloc[0]

# The worst symbol is the last item (index -1).
worst_symbol = sorted_symbols.index[-1]
worst_loss = sorted_symbols.iloc[-1]


# 3. Sort the profits in ASCENDING order.
#    This places the symbols with the largest losses at the top.
sorted_by_loss = symbol_profits.sort_values(ascending=True)


# 4. Select the first 10 rows from the sorted list using .head(10).
worst_10_symbols = sorted_by_loss.head(10)


# 5. Print the final result.
print("--- 10 Worst Performing Symbols (by Total Profit) ---")
print(worst_10_symbols)

#======================================



# --- NEW: Calculate and print the additional statistics ---

# Count the number of symbols with negative total profit (losers)
num_loss_making_symbols = (symbol_profits <= 0).sum()

# Count the number of symbols with positive total profit (winners)
num_profit_making_symbols = (symbol_profits > 0).sum()

# Calculate the ratio of losers to winners
# Add a check to prevent division by zero if there are no winners
if num_profit_making_symbols > 0:
    loss_to_profit_ratio = num_loss_making_symbols / num_profit_making_symbols
else:
    loss_to_profit_ratio = float('inf') # Indicate infinite ratio if no winners

# Print the new stats
print("\n--- Symbol Profit/Loss Summary ---")
print(f"Number of Profit-Making Symbols: {num_profit_making_symbols}")
print(f"Number of Loss-Making Symbols:   {num_loss_making_symbols}")
print(f"Ratio of Loss-Makers to Profit-Makers: {loss_to_profit_ratio:.2f}")




def find_max_daily_gmv(trade_log):

    """
    Calculates the maximum and mean Gross Merchandise Value (GMV) on a single day
    from a trade log DataFrame. The mean calculation accounts for all days in the
    period, including non-trading days.

    Args:
        trade_log (pd.DataFrame): DataFrame containing the trade log. 
                                  Index must be a datetime object ('entry_date').
                                  Must contain 'entry_price', 'exit_price', 
                                  'qty', and 'exit_date' columns.

    Returns:
        tuple: A tuple containing the max_gmv_value and the mean_gmv_value.
    """

    # Calculate the total value of each entry and exit transaction
    trade_log['entry_value'] = trade_log['entry_price'] * trade_log['qty']
    trade_log['exit_value'] = trade_log['exit_price'] * trade_log['qty']

    # Group all entry values by their calendar day and sum them up
    daily_entries = trade_log.groupby(trade_log.index.date)['entry_value'].sum()

    # Group all exit values by their calendar day and sum them up
    # CORRECTED: Group exits by 'exit_date', not the index ('entry_date')
    daily_exits = trade_log.groupby(trade_log['exit_date'].dt.date)['exit_value'].sum()

    # Combine the daily entry and exit sums to get the total GMV for each day
    # This series only contains days where a trade occurred.
    daily_gmv = daily_entries.add(daily_exits, fill_value=0)

    # Find the maximum value. This can be done before reindexing for efficiency.
    if daily_gmv.empty:
        return 0, 0
    max_gmv_value = daily_gmv.max()

    # --- MODIFICATION TO ACCOUNT FOR NON-TRADING DAYS ---

    # 1. Determine the full date range of the backtest period.
    #    We use the earliest entry and latest exit to define the period.
    start_date = trade_log.index.min().date()
    end_date = trade_log['exit_date'].max().date()
    all_days_in_period = pd.date_range(start=start_date, end=end_date, freq='D')

    # 2. Reindex the 'daily_gmv' Series to include all days.
    #    This assigns a GMV of 0 to all non-trading days.
    daily_gmv = daily_gmv.reindex(all_days_in_period, fill_value=0)
    
    # --- END OF MODIFICATION ---

    # Now, this calculation correctly finds the mean over the entire period.
    mean_gmv_value = daily_gmv.mean()

    return max_gmv_value, mean_gmv_value


max_gmv , mean_gmv = find_max_daily_gmv(local_trade_log2)

print(f"Average daily GMV is {mean_gmv}")
print(f"Max daily gmv is {max_gmv}")


print(f"Return on average GMV is {local_trade_log2['profit'].sum()/mean_gmv}")
print(f"Return on average GMV is {local_trade_log2['profit'].sum()/max_gmv}")



# Create HTML report
html_report = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Backtest Performance Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; }}
        h2 {{ color: #2980b9; }}
        .metric-container {{ display: flex; flex-wrap: wrap; gap: 20px; }}
        .metric-card {{
            background: #f8f9fa;
            border: 1px solid #dee2e6;
            border-radius: 5px;
            padding: 15px;
            flex: 1 1 300px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .metric-title {{ font-weight: bold; color: #2c3e50; }}
        .metric-value {{ font-size: 1.2em; color: #27ae60; }}
        .negative {{ color: #e74c3c; }}
        .plot-container {{ margin: 30px 0; text-align: center; }}
        table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
        th, td {{ padding: 10px; text-align: left; border-bottom: 1px solid #ddd; }}
        th {{ background-color: #3498db; color: white; }}
        tr:nth-child(even) {{ background-color: #f2f2f2; }}
    </style>
</head>
<body>
    <h1>Strategy Backtest Performance Report</h1>

    <div class="plot-container">
        <h2>Equity Curve</h2>
        <img src="data:image/png;base64,{plot_data}" alt="Equity Curve">
    </div>

    <h2>Key Performance Metrics</h2>
    <div class="metric-container">
        <div class="metric-card">
            <div class="metric-title">Total Trades</div>
            <div class="metric-value">{total_trade_count:,}</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Win Rate</div>
            <div class="metric-value">{100 - losers_percentage:.1f}%</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Average Trade Profit</div>
            <div class="metric-value">${average_trade_profit:,.2f}</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Trade Expectancy</div>
            <div class="metric-value">${trade_expectancy:,.2f}</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Risk-Reward Ratio</div>
            <div class="metric-value">{risk_reward_ratio:.2f}:1</div>
        </div>
        <div class="metric-card">
            <div class="metric-title">Max Drawdown</div>
            <div class="metric-value negative">${max_unrealised_drawdown:,.2f}</div>
        </div>
    </div>

    <h2>Long/Short Breakdown</h2>
    <table>
        <tr>
            <th>Metric</th>
            <th>Long Trades</th>
            <th>Short Trades</th>
        </tr>
        <tr>
            <td>Count</td>
            <td>{num_long_trades:,}</td>
            <td>{num_short_trades:,}</td>
        </tr>
        <tr>
            <td>Win Rate</td>
            <td>{long_win_ratio:.1%}</td>
            <td>{short_win_ratio:.1%}</td>
        </tr>
        <tr>
            <td>Avg Profit (bps)</td>
            <td>{avg_long_profit_bps:.1f}</td>
            <td>{avg_short_profit_bps:.1f}</td>
        </tr>
        <tr>
            <td>Total Profit</td>
            <td>${profit_total_long:,.2f}</td>
            <td>${profit_total_short:,.2f}</td>
        </tr>
        <tr>
            <td>Max Win Streak</td>
            <td>{long_consecutive[long_consecutive['type']=='win']['length'].max() if len(long_consecutive) > 0 else 0}</td>
            <td>{short_consecutive[short_consecutive['type']=='win']['length'].max() if len(short_consecutive) > 0 else 0}</td>
        </tr>
        <tr>
            <td>Max Loss Streak</td>
            <td>{long_consecutive[long_consecutive['type']=='loss']['length'].max() if len(long_consecutive) > 0 else 0}</td>
            <td>{short_consecutive[short_consecutive['type']=='loss']['length'].max() if len(short_consecutive) > 0 else 0}</td>
        </tr>
    </table>

    <h2>Position Sizing</h2>
    <div class="metric-container">
        
        <div class="metric-card">
            <div class="metric-title">Max Entries Per Day</div>
            <div class="metric-value">{max_entries_per_day}</div>
        </div>
    </div>

    <h2>Trade Duration</h2>
    <div class="metric-container">
        <div class="metric-card">
            <div class="metric-title">Avg Holding Days</div>
            <div class="metric-value">{local_trade_log2['holding_days'].mean():.1f}</div>
        </div>
    </div>
</body>
</html>
"""

# Display the report
HTML(html_report)

# To save the report to a file
with open('Nasdaq_Vol_stocks.html', 'w') as f:
    f.write(html_report)

local_trade_log2.to_csv("Nasdaq_TL.csv", index=True)

# Convert total_pnl_series dict (timestamp → pnl) into DataFrame
mtm_equity_df = pd.DataFrame(list(total_pnl_series.items()), columns=["timestamp", "MTM_PnL"])

# Sort by timestamp (important to keep the curve ordered)
mtm_equity_df.sort_values("timestamp", inplace=True)

# Add equity column: starting capital + cumulative PnL
mtm_equity_df["Equity"] = intial_capital + mtm_equity_df["MTM_PnL"].cumsum()

# Set timestamp as index for convenience
mtm_equity_df.set_index("timestamp", inplace=True)

# Define full path
mtm_equity_file = os.path.join(trades_strategy_folder, "MTM_equity.csv")

# Save to CSV
mtm_equity_df.to_csv(mtm_equity_file)

print(f"✅ MTM equity curve saved at: {mtm_equity_file}")


for symbol, data in absolute_symbol_pnl_series.items():
    if not data:
        print(f"No data for {symbol}, skipping.")
        continue
    try:
        tdf = pd.DataFrame(data, columns=['timestamp', 'pnl_absolute'])
        tdf.set_index('timestamp', inplace=True)
        file_path = os.path.join(df_folder, f"{symbol}_pnl.csv")
        tdf.to_csv(file_path)
        print(f"Successfully saved {file_path}")
    except Exception as e:
        print(f"Error processing {symbol}: {e}")


import quantstats as qs
qs.extend_pandas()

qs.reports.basic(local_trade_log2['EC'])

# --- 2. Plotting the 'EC' column ---
plt.figure(figsize=(12, 6)) # Optional: Adjust the figure size

local_trade_log2['EC'].plot(grid=True) # The main plotting command

# --- 3. Add labels and a title for clarity ---
plt.title('EC Over Time')
plt.xlabel('Date')
plt.ylabel('EC Value')

# --- 4. Display the plot ---
plt.show()


In [ ]:

com_legs_df[[f"{legs}_intraday_low" for legs in legs_to_process]] =com_legs_df[[f"{legs}_bidPrice" for legs in legs_to_process]].apply(lambda x: intraday_low(com_legs_df, low_col=x.name,agg_func='min'))

com_legs_df[[f"{legs}_intraday_high" for legs in legs_to_process]] =com_legs_df[[f"{legs}_askPrice" for legs in legs_to_process]].apply(lambda x: intraday_high(com_legs_df, high_col=x.name,agg_func='max'))


# Calculate Today's Opening Price for each leg
com_legs_df[[f'{leg}_todayo' for leg in legs_to_process]] =  com_legs_df[[f'{leg}_close' for leg in legs_to_process]].resample('D').transform('first')

# Calculate Previous Day's Low (using bidPrice) for each leg
com_legs_df[[f"{leg}_prevdayl" for leg in legs_to_process]] = com_legs_df[[f"{leg}_bidPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_low_numba(com_legs_df, n=1, column=x.name))

# Calculate Previous Day's High (using askPrice) for each leg
com_legs_df[[f"{leg}_prevdayh" for leg in legs_to_process]] = \
    com_legs_df[[f"{leg}_askPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_high_numba(com_legs_df, n=1, column=x.name) )

close_cols = [f'{leg}_close' for leg in legs_to_process]
window_size = 15

# 1. Rolling 15-Bar Highest Close
com_legs_df[[f'{leg}_15min_high' for leg in legs_to_process]] = \
    com_legs_df[close_cols].rolling(window=window_size).max()

# 2. Rolling 15-Bar Lowest Close
com_legs_df[[f'{leg}_15min_low' for leg in legs_to_process]] = \
    com_legs_df[close_cols].rolling(window=window_size).min()

# 3. Rolling 15-Bar Mean Close
com_legs_df[[f'{leg}_15min_mean' for leg in legs_to_process]] = \
    com_legs_df[close_cols].rolling(window=window_size).mean()


com_legs_df[[f'{leg}_ROC_5' for leg in legs_to_process]] = com_legs_df[[f"{leg}_close" for leg in legs_to_process]].pct_change(periods=5)



In [9]:
# ==============================================================================
# ---  HELPER FUNCTION for Clean and Fast Trade Logging ---
# ==============================================================================
def log_and_close_trade(trade_log_list, leg, exit_timestamp, exit_price, exit_reason, state_vars):
    """
    Calculates P&L, appends the trade to a log list, and resets the state for that leg.
    """
    # Unpack state variables for easier access
    entry_dates, entry_prices, qty, position_type, instrument_positions, bar_count = (
        state_vars['entry_dates'], state_vars['entry_prices'], state_vars['qty'],
        state_vars['position_type'], state_vars['instrument_positions'], state_vars['bar_count']
    )
    
    if entry_prices[leg] is None: return # Avoid logging if no entry price exists

    if position_type[leg] == 1:  # Long position
        profit = (exit_price - entry_prices[leg]) * qty[leg]
        side = 'BUY'
    else: # Short position
        profit = (entry_prices[leg] - exit_price) * qty[leg]
        side = 'SELL'

    trade_log_list.append({
        'name': leg, 'Side': side, 'entry_date': entry_dates[leg],
        'entry_price': entry_prices[leg], 'qty': qty[leg],
        'exit_date': exit_timestamp, 'exit_price': exit_price,
        'profit': profit, 'exit_reason': exit_reason
    })

    # Reset state variables
    instrument_positions[leg] = False
    entry_prices[leg] = None
    bar_count[leg] = 0
    
    print(f"   -> EXIT {leg} ({side}) at {exit_price:.2f} for {exit_reason}. P&L: ${profit:.2f}")

# ==============================================================================

In [ ]:
strike_interval = 1
lookback_days = 3
option_multiplier = 100
from collections import  defaultdict

# ==============================================================================
# SCRIPT STARTS HERE
# ==============================================================================

trade_log = []
nyse = mcal.get_calendar('NYSE') 
unique_days = com_df.index.normalize().unique()
print(f"🚀 Starting backtest for {len(unique_days)} unique days...")

intial_capital = 100000
position_cap = 100000

# Track local trade log
local_trade_log2 = pd.DataFrame(columns=['name', 'Side', 'entry_date', 'entry_price' , 'qty' ,  'exit_date', 'exit_price', 'order_no','profit' ])

# ✅ Initialize a list to hold all trade dictionaries.
final_trade_log_list = []

# --- Main Loop: Iterate Through Each Day ---
for day in unique_days[2:]:
    
    day_str = day.strftime('%Y-%m-%d')
    print(f"\n--- Processing Day: {day_str} ---")

    try:
        
        daily_underlying_df = com_df[com_df.index.date == day.date()]
        if daily_underlying_df.empty: continue

        open_price = daily_underlying_df['SPY_close'].iloc[0]
        if pd.isna(open_price): continue
        
        atm_strike = np.round(open_price)
        #print(f"  STEP 1: ATM Strike for the day set to {atm_strike}")

        # --- Dynamically Build and Fetch Data Based on Config ---
        all_legs_hist_data = {}
        fetch_success = True
        legs_to_trade = []

        for leg_name, params in strategy_config.items():
            
            target_strike = atm_strike + (params['strike_offset'] * strike_interval)

            print(f"Atm strike is {atm_strike} and taegt strike is {target_strike} and optiontyoe is {params['option_type']}")
            schedule = nyse.schedule(start_date=day, end_date=day + pd.Timedelta(days=14))
            target_expiry_str = schedule.index[params['expiry_offset']].strftime('%Y-%m-%d')
            print(f"Traget expiry for {leg_name} and {day_str} is {target_expiry_str} ")
            legs_to_trade.append({
                'name': leg_name, 'strike': target_strike,
                'expiry': target_expiry_str, 'type': params['option_type']})
            
            leg_hist_df = fetch_option_data_for_n_days(
                con=con, ticker='SPY', end_date_str=day_str, n_days=lookback_days,
                strike=target_strike, expiry_str=target_expiry_str, option_type=params['option_type'] )
                    
            # ✅ ADD THIS LINE TO DEBUG:
            #print(f"Columns found for {leg_name}: {leg_hist_df.columns.tolist()}")

            if not leg_hist_df.empty:
                # Find rows where both bidPrice and askPrice are 0
                zero_rows = leg_hist_df[ (leg_hist_df['bidPrice'] == 0) & (leg_hist_df['askPrice'] == 0) ]
                zero_count = len(zero_rows)
                total_rows = len(leg_hist_df)
                
                #print(f"  📊 {leg_name}: {zero_count}/{total_rows} rows have both bid & ask = 0")
                # One-liner: Replace zeros with NaN and forward fill for both columns
                leg_hist_df[['bidPrice', 'askPrice']] = leg_hist_df[['bidPrice', 'askPrice']].replace(0, np.nan).ffill()
                
                if zero_count > 1:
                    print(f"  ⚠️  {zero_count} Rows with zero prices for {leg_name}:")

                    if leg_hist_df.empty:
                        print(f"  ⚠️ Could not fetch data for {leg_name}. Skipping day.")
                        fetch_success = False
                        break
                all_legs_hist_data[leg_name] = leg_hist_df
            else:
                print(f"No options data found for {date_str}")
        if not fetch_success: 
            print(f"Not able to fetch options for {day} ")
            continue

        # --- Merge and Prepare Data ---
        prefixed_dfs_today = []
        for leg_name, leg_hist_df in all_legs_hist_data.items():
            
            prefixed_dfs_today.append(leg_hist_df.add_prefix(f'{leg_name}_'))
       
        # 1. First, concatenate all the prefixed option leg DataFrames together
        com_legs_df = pd.concat(prefixed_dfs_today, axis=1, join='outer')
        com_legs_df = com_legs_df[com_legs_df.index.time>time(9 , 30)]
        
        if ( len(com_legs_df)<10):
            print("\n\n =============")
            print(f"Leg data very less for {date_str}")


        #==== now do all calculations and indicators on this
        legs_to_process = ['Leg1', 'Leg2']
                
        com_legs_df[[f"{legs}_intraday_low" for legs in legs_to_process]] = com_legs_df[[f"{legs}_bidPrice" for legs in legs_to_process]].apply(lambda x: intraday_low(com_legs_df, low_col=x.name,agg_func='min'))

        com_legs_df[[f"{legs}_intraday_high" for legs in legs_to_process]] = com_legs_df[[f"{legs}_askPrice" for legs in legs_to_process]].apply(lambda x: intraday_high(com_legs_df, high_col=x.name,agg_func='max'))


        # Calculate Today's Opening Price for each leg
        com_legs_df[[f'{leg}_todayo' for leg in legs_to_process]] =  com_legs_df[[f'{leg}_close' for leg in legs_to_process]].resample('D').transform('first')

        # Calculate Previous Day's Low (using bidPrice) for each leg
        com_legs_df[[f"{leg}_prevdayl" for leg in legs_to_process]] = com_legs_df[[f"{leg}_bidPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_low_numba(com_legs_df, n=1, column=x.name))

        # Calculate Previous Day's High (using askPrice) for each leg
        com_legs_df[[f"{leg}_prevdayh" for leg in legs_to_process]] = \
            com_legs_df[[f"{leg}_askPrice" for leg in legs_to_process]].apply(lambda x: get_x_day_high_numba(com_legs_df, n=1, column=x.name) )

        close_cols = [f'{leg}_close' for leg in legs_to_process]
        window_size = 15

        # # 1. Rolling 15-Bar Highest Close
        # com_legs_df[[f'{leg}_15min_high' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).max()

        # # 2. Rolling 15-Bar Lowest Close
        # com_legs_df[[f'{leg}_15min_low' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).min()

        # # 3. Rolling 15-Bar Mean Close
        # com_legs_df[[f'{leg}_15min_mean' for leg in legs_to_process]] = \
        #     com_legs_df[close_cols].rolling(window=window_size).mean()

        com_legs_df[[f'{leg}_ROC_5' for leg in legs_to_process]] = com_legs_df[[f"{leg}_close" for leg in legs_to_process]].pct_change(periods=5)
#=== merge underlying back with com_legs_df

        com_df3 = pd.concat([daily_underlying_df , com_legs_df] , join='outer' , axis=1)

        # --- FIX: Add this line to filter for the current day ---
        com_df3 = com_df3[com_df3.index.date == pd.to_datetime(day_str).date()]

         # Initialize state for this day's trading (if needed)
        instrument_positions = {leg: False for leg in legs_to_process}
        entry_prices = {leg: None for leg in legs_to_process}
        # ... (add any other state variables you need: qty, entry_dates, etc.) ...
       
        entry_dates = {leg: None for leg in legs_to_process}
                
        # To track the current drawdown value for each active leg
        drawdown = {leg: None for leg in legs_to_process}

        # To track P&L for each leg (can be redundant if using series)
        symbol_pnl = {leg: None for leg in legs_to_process}

        # To track the maximum drawdown seen for each leg
        max_drawdown = {leg: 0 for leg in legs_to_process}

        # To count how many bars an active position has been held
        bar_count = {leg: 0 for leg in legs_to_process}

        # To store the quantity/number of contracts for each leg
        qty = {leg: 0 for leg in legs_to_process}

        # To store an order identifier for each leg's trade
        order_no = {leg: 0 for leg in legs_to_process}

        # To store the long/short direction (e.g., 1 for long, -1 for short)
        position_type = {leg: 0 for leg in legs_to_process}

        # To store the drawdown history for each leg
        drawdown_series = {leg: [] for leg in legs_to_process}

        # To store the P&L history for each leg
        symbol_pnl_series = {leg: [] for leg in legs_to_process}

        # To store another P&L history (can be consolidated with the one above)
        absolute_symbol_pnl_series = {leg: [] for leg in legs_to_process}
        # ===================================================================
        # --- Bar-by-Bar Simulation Loop using for i in range() ---
        # ===================================================================
        daily_trade = {'legs': {}}
        print("  STEP 3: Starting bar-by-bar simulation...")
        
        logger.info(f"Running analysis on: {intial_capital} of capital")
        cumulative_realized_pnl = 0.0


        # === Exposure & portfolio trackers ===
        exposure = 0.0        # net exposure
        gross_exposure = 0.0  # absolute exposure

        exposure_series = {}          # net exposure at each timestamp
        gross_exposure_series = {}    # absolute exposure
        portfolio_value_series = {}   # portfolio equity curve
        
        # Initialize before main loop
        total_pnl_series = defaultdict(float)
        # Process each row in time order

        # --- 2C. INITIALIZE STATE FOR THE DAY ---
        state_vars = {
            'instrument_positions': {leg: False for leg in legs_to_process},
            'entry_prices': {leg: None for leg in legs_to_process},
            'entry_dates': {leg: None for leg in legs_to_process},
            'bar_count': {leg: 0 for leg in legs_to_process},
            'qty': {leg: 0 for leg in legs_to_process},
            'position_type': {leg: 0 for leg in legs_to_process}
        }

        # --- 2D. BAR-BY-BAR SIMULATION FOR THE CURRENT DAY ---
        print("   STEP 3: Starting bar-by-bar simulation...")
        
        for i in range(1, len(com_df3)):
            
            row = com_df3.iloc[i]
            timestamp = com_df3.index[i]

            position_is_open = state_vars['instrument_positions']['Leg1'] # Check one leg to represent the spread

            # --- CHECK FOR EXITS FIRST ---
            if position_is_open:
                state_vars['bar_count']['Leg1'] += 1
                state_vars['bar_count']['Leg2'] += 1
                
                # Exit Condition Example: End of day
                if timestamp.time() >= time(15, 45) and state_vars['bar_count']['Leg1']>1  :
                    print(f"   -> Exiting spread at {timestamp.time()} for End of Day.")
                    # Close both legs
                    leg1_exit_price = row['Leg1_bidPrice'] if state_vars['position_type']['Leg1'] == 1 else row['Leg1_askPrice']
                    leg2_exit_price = row['Leg2_bidPrice'] if state_vars['position_type']['Leg2'] == 1 else row['Leg2_askPrice']
                    
                    log_and_close_trade(final_trade_log_list, 'Leg1', timestamp, leg1_exit_price, "EOD", state_vars)
                    log_and_close_trade(final_trade_log_list, 'Leg2', timestamp, leg2_exit_price, "EOD", state_vars)
                    break # Day is over, exit simulation loop

                if timestamp.time() >= time(9, 45) and state_vars['bar_count']['Leg1']>5 and row['Leg1_bidPrice']>state_vars['entry_prices']['Leg1']+0.5  :
                    print(f"   -> Exiting spread at {timestamp.time()} for End of Day.")
                    # Close both legs
                    leg1_exit_price = row['Leg1_bidPrice'] if state_vars['position_type']['Leg1'] == 1 else row['Leg1_askPrice']
                    leg2_exit_price = row['Leg2_bidPrice'] if state_vars['position_type']['Leg2'] == 1 else row['Leg2_askPrice']
                    
                    log_and_close_trade(final_trade_log_list, 'Leg1', timestamp, leg1_exit_price, "EOD", state_vars)
                    log_and_close_trade(final_trade_log_list, 'Leg2', timestamp, leg2_exit_price, "EOD", state_vars)
                    print(f"Stop loss hit")
                    break # Day is over, exit simulation loop
        


            # --- THEN, CHECK FOR ENTRIES ---
            else: # if not position_is_open
                
                # Define your entry condition for the entire spread
                entry_condition_met = (
                    time(9, 40) <= timestamp.time() <= time(13, 15) and  row[f"Leg1_close"]<1.5 and row[f"Leg1_close"]>1    and com_df3[f"SPY_3d_Pctl"].iloc[i-1]<0.5 and  com_df3[f"SPY_3d_Pctl"].iloc[i-1]>0  and row[f"Leg1_close"]<min((com_df3[f"Leg1_intraday_high"].iloc[i - 1]*0.9) , com_df3[f"Leg1_close"].iloc[i - 1]*0.999 )  and com_df3[f"Leg1_ROC_5"].iloc[i - 1]>0.01
                    
                )

                if entry_condition_met:
                    print(f"   -> ENTRY signal for spread triggered at {timestamp.time()}")
                    
                    # --- Open Position for BOTH LEGS ---
                    for leg_name, params in strategy_config.items():
                        if params['action'] == 'BUY':
                            entry_price = row[f"{leg_name}_askPrice"]
                            pos_type = 1
                        else: # SELL
                            entry_price = row[f"{leg_name}_bidPrice"]
                            pos_type = -1
                        # --- ✅ Set quantity based on leg_name inside the loop ---
                        if leg_name == 'Leg1':
                            trade_qty = 100  # Set quantity for Leg1
                        elif leg_name == 'Leg2':
                            trade_qty = 100  # Set quantity for Leg2
                        
                        # Update state for the current leg
                        state_vars['instrument_positions'][leg_name] = True
                        state_vars['entry_prices'][leg_name] = entry_price
                        state_vars['entry_dates'][leg_name] = timestamp
                        state_vars['position_type'][leg_name] = pos_type
                        state_vars['qty'][leg_name] = trade_qty
                        state_vars['bar_count'][leg_name] = 1
                        print(f"      -> Executing {leg_name} ({params['action']}) at {entry_price:.2f}")

    except Exception as e:
        print(f"   ❌ An error occurred on {day_str}: {e}")

# --- 3. FINAL RESULTS ANALYSIS ---
print("\n--- BACKTEST COMPLETE ---")
if final_trade_log_list:
    log_df = pd.DataFrame(final_trade_log_list)
    print(log_df)
else:
    print("No trades were logged.")

In [ ]:
log_df['profit'].cumsum().plot()

In [ ]:
log_df

In [ ]:
# Step 2: Group by the date of the 'entry_date' and sum the 'profit' column
daily_profit = log_df.groupby(log_df['entry_date'].dt.date)['profit'].sum()

In [ ]:
daily_profit.cumsum().plot()